In [1]:
%load_ext sparkmagic.magics
from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy() 
dsxhi_util.list_livy_endpoints()

success configuring sparkmagic livy.
['https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1']


In [2]:
%%spark config
{"executorCores": 2, "numExecutors": 10, "executorMemory": "15g", 
 "driverMemory": "12g", "proxyUser": "aliu-", "driverCores": 1, 
 "conf": {"spark.yarn.appMasterEnv.THEANO_FLAGS": "base_compiledir=${PWD}/.theano"}}

In [3]:
%spark add -s inactivePurchaseLeadGen -k -l python -u https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
20985,application_1582863878751_121765,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [4]:
%%spark

import pyspark
import os, sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import lower
import pyspark.sql.functions as F
from pyspark.sql.functions import last_day, rand, dense_rank, last_day, col, size, length, when, upper, unix_timestamp, avg, substring, lower, udf, sum, count, lit, mean, concat, countDistinct, desc, from_unixtime, row_number, year, month, to_date, upper, months_between
from pyspark.sql import DataFrameStatFunctions as statFunc
from pyspark.sql.types import *
from pyspark.sql.window import Window
from functools import reduce

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.feature import VectorIndexer, VectorAssembler, StringIndexer, QuantileDiscretizer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from os.path import expanduser, join, abspath
import time
import pandas as pd

spark = SparkSession.builder.getOrCreate()

In [5]:
%%spark

model_save_path = '/dev/projects/retention_models/purchase_payoff/models/'
training_data_path = '/dev/projects/retention_models/purchase_payoff/training/data/train/'
test_data_path = '/dev/projects/retention_models/purchase_payoff/training/data/test/'
result_path = '/dev/projects/retention_models/purchase_payoff/training/results/mover_12/'

monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'

# 1. Load Preprocess Data

### Data for Mover Purchase V1.1 6-12

In [6]:
%%spark

df201612 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201612.parquet').drop('gcid')
df201703 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201703.parquet').drop('gcid')
df201706 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201706.parquet').drop('gcid')
df201709 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201709.parquet').drop('gcid')
df201712 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201712.parquet').drop('gcid')
df201803 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201803.parquet').drop('gcid')
df201806 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201806.parquet').drop('gcid')
df201809 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201809.parquet').drop('gcid')
df201812 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201812.parquet')
df201903 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201903.parquet')
df201906 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201906.parquet')
df201909 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201909.parquet')
df201912 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201912.parquet')
df202003 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202003.parquet')


df_preprocessed = df201612.union(df201703).union(df201706).union(df201709).union(df201712)\
                            .union(df201803).union(df201806).union(df201809).union(df201812)\
                            .union(df201903).union(df201906).union(df201909).union(df201912)\
                            .union(df202003)\
                            .drop('borrowerage_bucket','p_married_exp','p_31_50_exp')

In [7]:
%%spark

actual = spark.read.parquet(actual_path + 'Actual_payoff_20200505.parquet')\
                .select('servicecalendardate', 'ln_no', 'purchasepayoff_6_12')

actual.show(1)

+-------------------+----------+-------------------+
|servicecalendardate|     ln_no|purchasepayoff_6_12|
+-------------------+----------+-------------------+
|         2016-10-31|1232492399|                0.0|
+-------------------+----------+-------------------+
only showing top 1 row

In [8]:
%%spark

target = 'purchasepayoff_6_12'

In [9]:
%%spark

df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()

In [10]:
%%spark

df_all.groupby('servicecalendardate').agg(count('ln_no'), sum(target)).orderBy(col('servicecalendardate').desc()).show()

+-------------------+------------+------------------------+
|servicecalendardate|count(ln_no)|sum(purchasepayoff_6_12)|
+-------------------+------------+------------------------+
|         2020-03-31|     1867297|                     0.0|
|         2019-12-31|     1802243|                     0.0|
|         2019-09-30|     1763391|                    42.0|
|         2019-06-30|     1822001|                 10724.0|
|         2018-12-31|     1726017|                 31292.0|
|         2018-09-30|     1683583|                 39172.0|
|         2018-06-30|     1619695|                 33992.0|
|         2018-03-31|     1577779|                 24945.0|
|         2017-12-31|     1530004|                 27817.0|
|         2017-09-30|     1468441|                 34156.0|
|         2017-06-30|     1423608|                 29665.0|
|         2017-03-31|     1379094|                 22382.0|
|         2016-12-31|     1326375|                 24884.0|
+-------------------+------------+------

# 3. Build Model

### Under-sample negative values

In [11]:
%%spark

# create fundtion for undersampling
def data_split(df, train_months, target, rate):
    
    ## Split Train/Validation From Test ## 
    train_validate = df.where(col('servicecalendardate').isin(train_months))
    
    ## Split Out Payoffs into Train/Validate ##
    train_refi_all = train_validate.filter(train_validate[target] == '1.')
    train_refi, validate_refi = train_refi_all.randomSplit([0.7, 0.3], seed=123)

    ## Split Out Non-Payoffs to Train/Validate
    train_nonrefi_all = train_validate.filter(train_validate[target] == '0.')
    train_nonrefi, validate_nonrefi = train_nonrefi_all.randomSplit([0.7, 0.3], seed=123)

    ## Undersample Non-Payoffs ## 
    train_non, notused1 = train_nonrefi.randomSplit([rate, 1-rate], seed=123)
    val_non, notused2 = validate_nonrefi.randomSplit([rate, 1-rate], seed=123)

    ## Create Final Training/Validate Sets
    train = train_refi.unionAll(train_non)
    validate = validate_refi.unionAll(val_non)
    train.cache()
    
    return train, validate

In [12]:
%%spark

train_months = ['2017-06-30', '2017-09-30', '2017-12-31', '2018-03-31', '2018-06-30', '2018-09-30']

df_train, df_validate = data_split(df_all, train_months, target, 0.05)

In [13]:
%%spark

df_train.groupby(['servicecalendardate', target]).count().orderBy('servicecalendardate', target).show()

+-------------------+-------------------+-----+
|servicecalendardate|purchasepayoff_6_12|count|
+-------------------+-------------------+-----+
|         2017-06-30|                0.0|48641|
|         2017-06-30|                1.0|20803|
|         2017-09-30|                0.0|50061|
|         2017-09-30|                1.0|24034|
|         2017-12-31|                0.0|52466|
|         2017-12-31|                1.0|19483|
|         2018-03-31|                0.0|54495|
|         2018-03-31|                1.0|17388|
|         2018-06-30|                0.0|55814|
|         2018-06-30|                1.0|23798|
|         2018-09-30|                0.0|57806|
|         2018-09-30|                1.0|27639|
+-------------------+-------------------+-----+

### Create Pipeline

In [14]:
%%spark

def create_pipeline(target, *arg):
    
    exclude_cols = ('servicecalendardate', 'ln_no', 'og_note_dt', 'mosaic_group_pur_exp', 'ratespread_min_pur_exp',
                'ln_purpose_type', 'investornameshort', 'ln_ann_int_rt', target) 
    cat_cols = [i[0] for i in df_train.dtypes if ((i[1]=='string') & (~i[0].endswith(exclude_cols)))]
    num_cols = [i[0] for i in df_train.dtypes if ((i[1].startswith(('int', 'double'))) & (~i[0].endswith(exclude_cols)))]
    
    stages = []
    
    for col in cat_cols:
        
        #Category indexing with StringIndexer
        indexer = StringIndexer(inputCol = col, outputCol = col+'_idx').setHandleInvalid('keep')
        stages += [indexer]
        
    #assemblerInputs = [c+'_vec' for c in cat_cols] + num_cols
    assemblerInputs = [c+'_idx' for c in cat_cols] + num_cols
    assembler = VectorAssembler(inputCols = assemblerInputs, outputCol = 'vectFeatures')
    
    stages += [assembler]
    
    lr = LogisticRegression(maxIter=100, regParam=0.1, elasticNetParam=0.0, fitIntercept = True,
                            featuresCol='vectFeatures', labelCol=target)
    rf = RandomForestClassifier(numTrees=250, maxDepth = 5, featuresCol='vectFeatures', labelCol=target)
    gbt = GBTClassifier(maxIter=100, featuresCol='vectFeatures', labelCol=target)

    pipeline_lr = Pipeline(stages = stages + [lr])
    pipeline_rf = Pipeline(stages = stages + [rf])
    pipeline_gbt = Pipeline(stages = stages + [gbt])
 
    if "lr" in arg and "rf" in arg and "gbt" in arg:
        return lr, rf, gbt, pipeline_lr, pipeline_rf, pipeline_gbt
    elif "lr" in arg and "rf" in arg:
        return lr, rf, pipeline_lr, pipeline_rf
    elif "lr" in arg and "gbt" in arg:
        return lr, gbt, pipeline_lr, pipeline_gbt 
    elif "rf" in arg and "gbt" in arg:
        return rf, gbt, pipeline_rf, pipeline_gbt
    elif "rf" in arg:
        return rf, pipeline_rf
    elif "lr" in arg:
        return lr, pipeline_lr
    elif "gbt" in arg:
        return gbt, pipeline_gbt
    else:
        return gbt, pipeline_gbt

In [15]:
%%spark

def fit_pipeline(pipeline, training_dataset):
    model_pipeline = pipeline.fit(training_dataset)
    return model_pipeline
    
def persist_modelpersist_(pipeline_model, model_name, model_save_path=model_save_path):
    pipeline_model.save(model_save_path + model_name)

In [16]:
%%spark

lr, rf, gbt, pipeline_lr, pipeline_rf, pipeline_gbt = create_pipeline(target, "lr", "rf", "gbt")

In [17]:
%%spark

# Logistic Regression
start = time.time()
model_pipeline_lr = fit_pipeline(pipeline_lr, df_train)
end = time.time()

print('Logistic Regression Training Time:', end - start)

Logistic Regression Training Time: 226.72631573677063

In [18]:
%%spark

# Random Forest
start = time.time()
model_pipeline_rf = fit_pipeline(pipeline_rf, df_train)
end = time.time()

print('Random Forest Training Time:', end - start)

Random Forest Training Time: 281.80665040016174

In [19]:
%%spark

# Gradient Boosting
start = time.time()
model_pipeline_gbt = fit_pipeline(pipeline_gbt, df_train)
end = time.time()

print('Gradient Boosting Training Time:', end - start)

Gradient Boosting Training Time: 3686.442395925522

### Save Models

In [20]:
%%spark

persist_modelpersist_(model_pipeline_lr, 'Mover_6to12_V1_1_LR_20200516')
persist_modelpersist_(model_pipeline_rf, 'Mover_6to12_V1_1_RF_20200516')
persist_modelpersist_(model_pipeline_gbt, 'Mover_6to12_V1_1_GBT_20200516')

### Read Models

In [14]:
%%spark

model_pipeline_lr = PipelineModel.load(path = model_save_path+'Mover_6to12_V1_1_LR_20200516')
model_pipeline_rf = PipelineModel.load(path = model_save_path+'Mover_6to12_V1_1_RF_20200516')
model_pipeline_gbt = PipelineModel.load(path = model_save_path+'Mover_6to12_V1_1_GBT_20200516')

# 3. Variable Importance

In [22]:
%%spark
def extractFeatureImp(model_pipeline, df_train, featuresCol):
    
    featureImp = model_pipeline.stages[-1].featureImportances
    transformed = model_pipeline.transform(df_train)
    list_extract = []
    
    for i in transformed.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + transformed.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
        
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    
    return(varlist.sort_values('score', ascending = False))
    return varlist

In [23]:
%%spark

gbt_feature_importance = extractFeatureImp(model_pipeline_gbt, df_train, "vectFeatures")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(gbt_feature_importance)

    idx                         name  \
5    13                 ageinmon_exp   
7    15              currentcltv_exp   
10   18           LiveYears_long_exp   
6    14                og_mtg_am_exp   
8    16                orig_fico_exp   
4    12           ratespread_min_exp   
1     9               home_value_exp   
17    3    mosaic_group_refi_exp_idx   
20    6      ln_purpose_type_exp_idx   
13   21                    ln_tr_exp   
0     8                   ct_age_exp   
3    11       personnum_per_room_exp   
21    7  og_occupy_stat_type_exp_idx   
9    17          LiveYears_short_exp   
19    5     loanamortizationtype_idx   
2    10                 p_edu_hs_exp   
18    4  loantypedescription_exp_idx   
16    2                 ct_3_exp_idx   
12   20         issingleborrower_exp   
11   19            LiveYears_grp_exp   
15    1                 ct_2_exp_idx   
14    0                 ct_1_exp_idx   

                                                 vals     score  
5            

# 4. ROC

### Predictions

In [15]:
%%spark

def predict(model_pipeline, data):
    
    _scoreUdf = udf(lambda v: float(v[1]), DoubleType())
    
    prediction = model_pipeline.transform(data)
    pred_df = prediction.withColumn('pred', _scoreUdf(prediction['probability']))
    return pred_df

### ROC

In [16]:
%%spark

def roc_auc(df, target):   
    
    pred_df_lr = predict(model_pipeline_lr, df)
    pred_df_rf = predict(model_pipeline_rf, df)
    pred_df_gbt = predict(model_pipeline_gbt, df)

    ### Calculate ROC ###  
    evalPred_lr = pred_df_lr.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')
    evalPred_rf = pred_df_rf.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')
    evalPred_gbt = pred_df_gbt.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')

    evaluatorLR = BinaryClassificationEvaluator()
    evaluatorRF = BinaryClassificationEvaluator()
    evaluatorGBT = BinaryClassificationEvaluator()

    print("Test Area Under ROC - LR: " + str(evaluatorLR.evaluate(evalPred_lr, {evaluatorLR.metricName: "areaUnderROC"})))        
    print("Test Area Under ROC - RF: " + str(evaluatorRF.evaluate(evalPred_rf, {evaluatorRF.metricName: "areaUnderROC"})))        
    print("Test Area Under ROC - GBT: " + str(evaluatorGBT.evaluate(evalPred_gbt, {evaluatorGBT.metricName: "areaUnderROC"})))
    print("\n")

In [17]:
%%spark

test_months = ['2016-12-31', '2017-03-31', '2018-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))

for df in [df_train, df_validate, df_test]:
    roc_auc(df, target)

Test Area Under ROC - LR: 0.6033404953545035
Test Area Under ROC - RF: 0.6223009772948884
Test Area Under ROC - GBT: 0.6570597923923083


Test Area Under ROC - LR: 0.604224026454145
Test Area Under ROC - RF: 0.6216724988467393
Test Area Under ROC - GBT: 0.6447669991316481


Test Area Under ROC - LR: 0.5919226796607918
Test Area Under ROC - RF: 0.6087913550884938
Test Area Under ROC - GBT: 0.6295103473346155

# 5. Capture Rate

In [18]:
%%spark

def deciles(df, model, target):
    
    pred_df = predict(model, df)

    pred_df = QuantileDiscretizer(numBuckets=10, inputCol="pred", outputCol="decile", relativeError=0.00001,
                             handleInvalid="error").fit(pred_df).transform(pred_df)
    pred_df = pred_df.withColumn('decile', (10 - F.col('decile')).cast('int'))
    
    window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
    total_target = pred_df.select(F.sum(target)).collect()[0][0]
    df_out = pred_df\
        .groupBy('decile', )\
        .agg(F.count('ln_no').alias('decile_cnt'), F.sum(target).alias('payoff_cnt'))\
        .withColumn('cum_sum', F.sum('payoff_cnt').over(window_cumsum) / total_target)\
        .sort('decile')

    return df_out

In [52]:
%%spark

date_label = test_months[2]
print(date_label)

2018-12-31

In [53]:
%%spark
print(date_label)
deciles(df_all.filter(col('servicecalendardate') == date_label), model_pipeline_lr, target).show()

2018-12-31
+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    172608|    4439.0|0.14185734373002684|
|     2|    172596|    4043.0| 0.2710596957688866|
|     3|    172591|    3668.0|0.38827815416080785|
|     4|    172559|    3435.0|0.49805061996676464|
|     5|    172645|    3441.0| 0.6080148280710724|
|     6|    172605|    3287.0| 0.7130576505177042|
|     7|    172606|    2894.0| 0.8055413524223444|
|     8|    172592|    2450.0| 0.8838361242490094|
|     9|    172609|    2053.0| 0.9494439473347821|
|    10|    172606|    1582.0|                1.0|
+------+----------+----------+-------------------+

In [54]:
%%spark
print(date_label)
deciles(df_all.filter(col('servicecalendardate') == date_label), model_pipeline_rf, target).show()

2018-12-31
+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    172604|    4848.0|0.15492777706762112|
|     2|    172595|    4113.0|0.28636712258724273|
|     3|    172593|    3589.0| 0.4010609740508756|
|     4|    172724|    3643.0| 0.5174805061996677|
|     5|    172468|    3650.0| 0.6341237376965358|
|     6|    172589|    3022.0| 0.7306979419659977|
|     7|    172612|    2617.0| 0.8143295410967659|
|     8|    172629|    2633.0| 0.8984724530231369|
|     9|    172587|    2030.0| 0.9633452639652307|
|    10|    172616|    1147.0|                1.0|
+------+----------+----------+-------------------+

In [55]:
%%spark
print(date_label)
deciles(df_all.filter(col('servicecalendardate') == date_label), model_pipeline_gbt, target).show()

2018-12-31
+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    172565|    5277.0|0.16863735139971878|
|     2|    172619|    4302.0|0.30611657931739744|
|     3|    172609|    4084.0| 0.4366291703949891|
|     4|    172628|    3596.0| 0.5515467212066982|
|     5|    172580|    3340.0| 0.6582832672887639|
|     6|    172602|    2970.0|  0.753195704972517|
|     7|    172604|    2692.0| 0.8392240828326729|
|     8|    172606|    2276.0| 0.9119583280071584|
|     9|    172605|    1825.0| 0.9702799437555925|
|    10|    172599|     930.0|                1.0|
+------+----------+----------+-------------------+

# 6. Predictions

In [ ]:
%%spark

def fullMonthPrediction(df):
    
    pred_df_lr = predict(model_pipeline_lr, df)
    validationPredictionsLR = pred_df_lr.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'logRegProb')
    
    pred_df_rf = predict(model_pipeline_rf, df)
    validationPredictionsRF = pred_df_rf.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'randForProb')
    
    pred_df_gbt = predict(model_pipeline_gbt, df)
    validationPredictionsGBT = pred_df_gbt.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'gbtProb')
    
    combinedFinalPred = validationPredictionsLR.join(validationPredictionsRF, on='ln_no', how='left')\
                                                .join(validationPredictionsGBT, on='ln_no', how='left')\
                                                .dropDuplicates()
    
    return combinedFinalPred

In [ ]:
%%spark

def pred_save(df, servicedate, filename, result_path = result_path):
    
    df1 = df.where(col('servicecalendardate') == servicedate)
    df_pred = fullMonthPrediction(df1)
    
    df_pred.coalesce(1).write.csv(result_path + filename, header=True)

In [ ]:
%%spark

def pred_save_newmonth(preprocessed_file, filename, monthly_prep_path=monthly_prep_path, result_path=result_path):
    
    df = spark.read.parquet(monthly_prep_path + preprocessed_file)
    df_pred = fullMonthPrediction(df)
    
    print(df.count())
    df_pred.coalesce(1).write.csv(result_path + filename, header=True)

In [ ]:
%%spark

# 2017-6
pred_save(df_all, '2017-06-30', 'pred_MoverV1_1_jun17_20200514.csv')

# 2017-9
pred_save(df_all, '2017-09-30', 'pred_MoverV1_1_sep17_20200514.csv')

# 2017-12
pred_save(df_all, '2017-12-31', 'pred_MoverV1_1_dec17_20200514.csv')

# 2018-3
pred_save(df_all, '2018-03-31', 'pred_MoverV1_1_mar18_20200514.csv')

# 2018-6
pred_save(df_all, '2018-06-30', 'pred_MoverV1_1_jun18_20200514.csv')

# 2018-9
pred_save(df_all, '2018-09-30', 'pred_MoverV1_1_sep18_20200514.csv')

# 2018-12
pred_save(df_all, '2018-12-31', 'pred_MoverV1_1_dec18_20200514.csv')

# 2019-03
pred_save(df_all, '2019-03-31', 'pred_MoverV1_1_mar19_20200514.csv')

# 2019-06
pred_save(df_all, '2019-06-30', 'pred_MoverV1_1_jun19_20200514.csv')

# 2019-09
pred_save(df_all, '2019-09-30', 'pred_MoverV1_1_sep19_20200514.csv')

In [ ]:
%%spark

# 2018-1
pred_save_newmonth('monthly_preprocessed_201801.parquet', 'pred_MoverV3_2_jan18_20190722.csv')

# 2018-2
pred_save_newmonth('monthly_preprocessed_201802.parquet', 'pred_MoverV3_2_feb18_20190722.csv')

# 2018-4
pred_save_newmonth('monthly_preprocessed_201804.parquet', 'pred_MoverV3_2_apr18_20190722.csv')

# 2018-5
pred_save_newmonth('monthly_preprocessed_201805.parquet', 'pred_MoverV3_2_may18_20190722.csv')

# 2018-7
pred_save_newmonth('monthly_preprocessed_201807.parquet', 'pred_MoverV3_2_jul18_20190722.csv')

# 2018-8
pred_save_newmonth('monthly_preprocessed_201808.parquet', 'pred_MoverV3_2_aug18_20190722.csv')

# 2018-10
pred_save_newmonth('monthly_preprocessed_201810.parquet', 'pred_MoverV3_2_oct18_20190722.csv')

# 2018-11
pred_save_newmonth('monthly_preprocessed_201811.parquet', 'pred_MoverV3_2_nov18_20190722.csv')

# 2019-1
pred_save_newmonth('monthly_preprocessed_201901.parquet', 'pred_MoverV3_2_jan19_20190722.csv')

# 2019-2
pred_save_newmonth('monthly_preprocessed_201902.parquet', 'pred_MoverV3_2_feb19_20190722.csv')

# 2019-4
pred_save_newmonth('monthly_preprocessed_201904.parquet', 'pred_MoverV3_2_apr19_20190722.csv')

# 2019-5
pred_save_newmonth('monthly_preprocessed_201905.parquet', 'pred_MoverV3_2_may19_20190722.csv')
